In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [19]:
df = pd.read_csv('../datasets/Churn_Modelling.csv')
df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0


In [22]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix

X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

col_transformer = ColumnTransformer(
    [('geography_encoded', OneHotEncoder(), [1])],
    remainder='passthrough'
)

X = col_transformer.fit_transform(X)
X = X[:, 1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [26]:
# Fitting XGBoost to the training set
from xgboost import XGBClassifier

classifier = XGBClassifier()
classifier.fit(X_train, y_train)

accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, n_jobs=-1)
print(f'k-fold mean accuracy: {accuracies.mean()}')
print(f'k-fold standard deviation: {accuracies.std()}')

cm = confusion_matrix(y_test, classifier.predict(X_test))
print(cm)

k-fold mean accuracy: 0.8625
k-fold standard deviation: 0.01017042280340401
[[1521   74]
 [ 197  208]]
